# Neo4j - Criação da base de dados "mockada"

In [48]:
%reload_ext neo4j_cypher_query

In [46]:
from neo4j import GraphDatabase

def setup_neo4j():
    """
    Connects to the Neo4j database, clears existing data,
    and creates a new graph structure with sample nodes and relationships.
    """
    uri = "neo4j://neo4j:7687"
    user = "neo4j"
    password = "strong_password"
    driver = None

    try:
        # --- 1. ESTABLISH CONNECTION ---
        driver = GraphDatabase.driver(uri, auth=(user, password))
        session = driver.session()
        print("Connection to Neo4j successful.")

        # --- 2. CLEAR EXISTING DATABASE (for a clean setup) ---
        print("Clearing existing database...")
        session.run("MATCH (n) DETACH DELETE n")
        print("Database cleared.")

        # --- 3. CREATE GRAPH SCHEMA AND DATA ---
        print("Creating graph nodes and relationships...")
        # This single, large query creates all nodes and relationships in one go.
        # MERGE is used to avoid creating duplicates if run multiple times.
        graph_query = """
// ===============================
// Apagar tudo 
// ===============================

MATCH (n)
DETACH DELETE n
        """
        session.run(graph_query)
        graph_query = """
// ===============================
// GÊNEROS
// ===============================
MERGE (:Genre {nome: "Ação"})
MERGE (:Genre {nome: "Drama"})
MERGE (:Genre {nome: "Comédia"})
MERGE (:Genre {nome: "Ficção"})
MERGE (:Genre {nome: "Suspense"})
        """
        session.run(graph_query)
        print("Graph data GÊNEROS created successfully.")
        graph_query = """
// ===============================
//  VÍDEOS
// ===============================
CREATE
  (v1:Video {id: "v1", titulo: "Matrix", tipo: "filme"}),
  (v2:Video {id: "v2", titulo: "Inception", tipo: "filme"}),
  (v3:Video {id: "v3", titulo: "Breaking Bad", tipo: "serie"}),
  (v4:Video {id: "v4", titulo: "Stranger Things", tipo: "serie"}),
  (v5:Video {id: "v5", titulo: "O Poderoso Chefão", tipo: "filme"}),
  (v6:Video {id: "v6", titulo: "Friends", tipo: "serie"}),
  (v7:Video {id: "v7", titulo: "The Office", tipo: "serie"}),
  (v8:Video {id: "v8", titulo: "John Wick", tipo: "filme"}),
  (v9:Video {id: "v9", titulo: "Dark", tipo: "serie"}),
  (v10:Video {id: "v10", titulo: "La Casa de Papel", tipo: "serie"}),
  (v11:Video {id: "v11", titulo: "Black Mirror", tipo: "serie"}),
  (v12:Video {id: "v12", titulo: "Sherlock", tipo: "serie"}),
  (v13:Video {id: "v13", titulo: "Vikings", tipo: "serie"}),
  (v14:Video {id: "v14", titulo: "Peaky Blinders", tipo: "serie"}),
  (v15:Video {id: "v15", titulo: "The Mandalorian", tipo: "serie"})

        """
        session.run(graph_query)
        print("Graph data VÍDEOS created successfully.")
        graph_query = """
        
// ===============================
//  ASSOCIAR VÍDEOS A GÊNEROS
// ===============================
MATCH (a:Genre {nome: "Ação"}), (d:Genre {nome: "Drama"}), (c:Genre {nome: "Comédia"}), 
      (f:Genre {nome: "Ficção"}), (s:Genre {nome: "Suspense"})
MATCH 
  (v1:Video {id: "v1"}), (v2:Video {id: "v2"}), (v3:Video {id: "v3"}), (v4:Video {id: "v4"}),
  (v5:Video {id: "v5"}), (v6:Video {id: "v6"}), (v7:Video {id: "v7"}), (v8:Video {id: "v8"}),
  (v9:Video {id: "v9"}), (v10:Video {id: "v10"}), (v11:Video {id: "v11"}), (v12:Video {id: "v12"}),
  (v13:Video {id: "v13"}), (v14:Video {id: "v14"}), (v15:Video {id: "v15"})
MERGE (v1)-[:HAS_GENRE]->(a)
MERGE (v2)-[:HAS_GENRE]->(f)
MERGE (v3)-[:HAS_GENRE]->(d)
MERGE (v4)-[:HAS_GENRE]->(f)
MERGE (v5)-[:HAS_GENRE]->(d)
MERGE (v6)-[:HAS_GENRE]->(c)
MERGE (v7)-[:HAS_GENRE]->(c)
MERGE (v8)-[:HAS_GENRE]->(a)
MERGE (v9)-[:HAS_GENRE]->(s)
MERGE (v10)-[:HAS_GENRE]->(a)
MERGE (v11)-[:HAS_GENRE]->(s)
MERGE (v12)-[:HAS_GENRE]->(s)
MERGE (v13)-[:HAS_GENRE]->(a)
MERGE (v14)-[:HAS_GENRE]->(d)
MERGE (v15)-[:HAS_GENRE]->(f);

        """
        session.run(graph_query)
        print("Graph data ASSOCIAR VÍDEOS A GÊNEROS created successfully.")
        graph_query = """
        
// ===============================
// USUÁRIOS
// ===============================
UNWIND range(1, 10) AS i
CREATE (:User {id: 'u' + i, nome: 'Usuario ' + i});

        """
        session.run(graph_query)
        print("Graph data USUÁRIOS created successfully.")
        graph_query = """
        
// ===============================
// VISUALIZAÇÕES ALEATÓRIAS
// ===============================
UNWIND range(1, 10) AS i
MATCH (u:User {id: 'u' + i})
WITH u
MATCH (v:Video)
WITH u, v, rand() AS r
WHERE r < 0.6
MERGE (u)-[:WATCHED {timestamp: timestamp()}]->(v);

        """
        session.run(graph_query)
        print("Graph data VISUALIZAÇÕES ALEATÓRIAS created successfully.")
        graph_query = """
        
// ===============================
//  CURTIDAS
// ===============================
MATCH (u:User)-[w:WATCHED]->(v:Video)
WITH u, v, rand() AS r
WHERE r < 0.3
MERGE (u)-[:LIKED]->(v);

        """
        session.run(graph_query)
        print("Graph data CURTIDAS created successfully.")
        graph_query = """
        
// ===============================
// AVALIAÇÕES (RATING)
// ===============================
MATCH (u:User)-[w:WATCHED]->(v:Video)
WITH u, v, rand() AS r
WHERE r < 0.5
WITH u, v, toInteger(1 + rand() * 5) AS nota
MERGE (u)-[:RATED {nota: nota}]->(v);

        """
        session.run(graph_query)
        print("Graph data AVALIAÇÕES (RATING) created successfully.")
        graph_query = """
        

// ===============================
// GÊNERO FAVORITO
// ===============================
MATCH (:User)-[r:FAVORITE_GENRE]->(:Genre)
DELETE r;

        """
        session.run(graph_query)
        graph_query = """
        
MATCH (u:User)-[:WATCHED]->(v:Video)-[:HAS_GENRE]->(g:Genre)
WITH u, g, count(*) AS qtd
ORDER BY qtd DESC
WITH u, collect({g: g, qtd: qtd}) AS generos

UNWIND generos AS item
WITH u, item.g AS g, item.qtd AS qtd, generos[0].qtd AS max_qtd
WHERE qtd = max_qtd
MERGE (u)-[:FAVORITE_GENRE]->(g);

        """
        session.run(graph_query)
        print("Graph data GÊNERO FAVORITO created successfully.")
        
        print("Graph data created successfully.")
        print("\nNeo4j setup is complete!")

    except Exception as e:
        print(f" Error during Neo4j setup: {e}")
    finally:
        if driver:
            driver.close()
            print("Connection to Neo4j is closed.")

# --- Execute the setup function ---
setup_neo4j()

Connection to Neo4j successful.
Clearing existing database...
Database cleared.
Creating graph nodes and relationships...
Graph data GÊNEROS created successfully.
Graph data VÍDEOS created successfully.
Graph data ASSOCIAR VÍDEOS A GÊNEROS created successfully.
Graph data USUÁRIOS created successfully.
Graph data VISUALIZAÇÕES ALEATÓRIAS created successfully.
Graph data CURTIDAS created successfully.
Graph data AVALIAÇÕES (RATING) created successfully.
Graph data GÊNERO FAVORITO created successfully.
Graph data created successfully.

Neo4j setup is complete!
Connection to Neo4j is closed.


# Visualizar a base de dados:

In [54]:
%%cypher

match (a)-[r]->(b)
match (c)

return *

GraphWidget(layout=Layout(height='800px', width='100%'))

# Recomendacao outros filmes e séries do tipo Quem viu, viu também para o usuário u1
##### OBS: Pode ser necessário trocar o u1 por u2 ... u10

In [92]:
%%cypher

MATCH (u1:User {id: "u1"})-[:WATCHED]->(v:Video)<-[:WATCHED]-(u2:User)-[:WATCHED]->(rec:Video)
WHERE u1 <> u2 AND NOT (u1)-[:WATCHED]->(rec)
RETURN DISTINCT rec AS recomendacao
LIMIT 5;


GraphWidget(layout=Layout(height='500px', width='100%'))

# Recomendação filmes e séries mais bem avaliados por todos os usuários (média da nota). 
##### Útil para a seção EM ALTA no protótipo.


In [101]:
%%cypher

MATCH (v:Video)<-[r:RATED]-()
RETURN v, avg(r.nota) AS media, count(*) AS votos
ORDER BY media DESC, votos DESC
LIMIT 5;


GraphWidget(layout=Layout(height='500px', width='100%'))

# Recomendação filmes e séries para o usuario u1 com base nos seus gêneros favoritos. 
##### Útil para a seção PARA VOCÊ  no protótipo.
##### OBS: Pode ser necessário trocar o u1 por u2 ... u10

In [104]:
%%cypher

MATCH (u:User {id: "u1"})-[:FAVORITE_GENRE]->(g:Genre)
MATCH (rec:Video)-[:HAS_GENRE]->(g)
WHERE NOT (u)-[:WATCHED]->(rec)
RETURN DISTINCT rec 
LIMIT 5;


GraphWidget(layout=Layout(height='500px', width='100%'))

# Recomendação de filmes e séries mais bem avaliados por todos os usuários (média da nota). 
##### Útil para a seção EM ALTA no protótipo.
##### OBS: Pode ser necessário trocar o u1 por u2 ... u10


In [106]:
%%cypher

MATCH (v:Video)<-[r:RATED]-()
RETURN v, avg(r.nota) AS media, count(*) AS votos
ORDER BY media DESC, votos DESC
LIMIT 5;


GraphWidget(layout=Layout(height='500px', width='100%'))

# Recomendação dos filmes mais assistidos nos últimos 30 dias por gênero. 
##### Útil para a seção EM ALTA no protótipo.
##### OBS: Pode ser necessário trocar o Ação por outro gênero


In [108]:
%%cypher

WITH timestamp() AS agora
MATCH (u:User)-[w:WATCHED]->(v:Video)-[:HAS_GENRE]->(g:Genre {nome: "Ação"})
WHERE w.timestamp >= agora - 1000 * 60 * 60 * 24 * 30
RETURN v, count(*) AS visualizacoes
ORDER BY visualizacoes DESC
LIMIT 10;


GraphWidget(layout=Layout(height='500px', width='100%'))